# Spark Structured Streaming - Demo
## Robotic Arm

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
import io
from pyspark.sql.functions import *
import time
import json
import struct
import requests 

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1,org.apache.spark:spark-streaming-kafka-0-10_2.11:2.4.5,org.apache.kafka:kafka-clients:2.6.0 pyspark-shell'
                                    
spark = (SparkSession.builder 
    .master("local[*]")
    .appName("test")
    .getOrCreate()
        )

spark

set up the environment variables

In [2]:
topic = 'RoboticArm'
servers = "kafka:9092"

## Answers in Spark Structured Streaming 

Please refer to [continuous-analytics-examples/blob/master/epl_robotic-arm/readme.md](https://github.com/quantiaconsulting/continuous-analytics-examples/blob/master/epl_robotic-arm/readme.md) for the EPL version of the following queries.

Let's first try with the model proposed for EPL and see what happens. To get the data run [datagen1.ipynb](datagen1.ipynb)

### Let's create the streaming Data Frames using the data in the kafka topic

In [3]:
from pyspark.sql.types import *

roboticArm_schema = StructType([
    StructField("id", StringType(), True),
    StructField("status", StringType(), True),
    StructField("stressLevel", IntegerType(), True),
    StructField("ts", TimestampType(), True)])

raw_roboticArm_df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("startingOffsets", "earliest")
  .option("subscribe", topic)
  .load())

roboticArm_sdf = (raw_roboticArm_df
                      .select(from_json(col("value").cast("string"), roboticArm_schema).alias("value"))
                      .select("value.*"))

In [4]:
roboticArm_sdf.printSchema()

root
 |-- id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- stressLevel: integer (nullable = true)
 |-- ts: timestamp (nullable = true)



### to make sure that it works, let's first inspect the content of the stream 

In [5]:
basic_query = (roboticArm_sdf
    .writeStream
    .format("memory") # this is for debug purpose only! DO NOT USE IN PRODUCTION
    .queryName("sinkTable")
    .start())

run the following cell to see the most recent content of the sinkTable

In [8]:
spark.sql("SELECT * FROM sinkTable ORDER BY TS DESC").show(5)

+---+-----------+-----------+-------------------+
| id|     status|stressLevel|                 ts|
+---+-----------+-----------+-------------------+
|  1|     moving|          2|2021-10-28 11:54:07|
|  2|     moving|          1|2021-10-28 11:54:07|
|  2|placingGood|          3|2021-10-28 11:53:59|
|  1|placingGood|          3|2021-10-28 11:53:59|
|  2| movingGood|          9|2021-10-28 11:53:49|
+---+-----------+-----------+-------------------+
only showing top 5 rows



do not forget to stop queries that you are not using

In [9]:
basic_query.stop()

# E1

> Propose how to model the streaming data generated by the robotic arms.

Let's first try with the model proposed for EPL and see what happens. 

# E2

> Write a continuous query that emits the max stress for each arm.

### the SQL sytyle

In [10]:
# create a logic table on top of the streaming data frame
roboticArm_sdf.createTempView("RoboticArm") # this time we will not clean it up, because we use it in the next queries

In [12]:
query_string = """
SELECT id, max(stressLevel) 
FROM RoboticArm 
GROUP BY id;
"""

# write your query in SQL, register it and start it
e2 = (spark.sql(query_string)
                     .writeStream
                     .format("memory")
                     .outputMode("complete") # <-- CHANGE HERE
                     .queryName("sinkTable")
                     .start())

In [15]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable").show(5) # woithout ORDER BY TS DESC because the result in the table is already only the most recent

+---+----------------+
| id|max(stressLevel)|
+---+----------------+
|  1|               7|
|  2|               9|
+---+----------------+



In [16]:
# clean up
e2.stop()

### The DataFrame style

In [26]:
# write your query in SQL, register it and start it
e2bis = (roboticArm_sdf
                     .groupBy("id")
                     .max()
                     .writeStream
                     .format("memory")
                     .outputMode("complete") # 
                     .queryName("sinkTable")
                     .start())

In [28]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable").show(5) # woithout ORDER BY TS DESC because the result in the table is already only the most recent

+---+----------------+
| id|max(stressLevel)|
+---+----------------+
|  1|               8|
|  2|               9|
+---+----------------+



In [29]:
# clean up
e2bis.stop()

# E3

> A continuous query that emits the average stress level between a pick (status==goodGrasped) and a place (status==placingGood).

Spark Structured Streaming does not support the EPL's operator `->` (that reads as *followed by*. We need to use a stream-to-stream join.

this is an hard task, let's simplify it

### E3.1

> A continuous query that emits the events between a moving (status==movingGood) and a place (status==placingGood)

NOTE: no request to average and only two consecutive events

In [31]:
# creating a streaming DataFrame with only event wher status='movingGood'
moving_sdf = (roboticArm_sdf
                .where("status='movingGood'")
                .withColumnRenamed("id","idMoving")
                .withColumnRenamed("ts","tsMoving")
               )

In [34]:
# creating a streaming DataFrame with only event wher status='placingGood'

placing_sdf = (roboticArm_sdf
                .where("status='placingGood'")
                .withColumnRenamed("id","idPlacing")
                .withColumnRenamed("ts","tsPlacing")
               )

Join without the event-time constraint

In [35]:
join_sdf = (moving_sdf.join(
  placing_sdf, expr("""
    (idMoving == idPlacing) AND
    (tsMoving < tsPlacing )
    """
    )))

In [36]:
e3 = (join_sdf
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

In [41]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsPlacing DESC").show(5,False) # note, I change ts in tsPlacing

+--------+----------+-----------+-------------------+---------+-----------+-----------+-------------------+
|idMoving|status    |stressLevel|tsMoving           |idPlacing|status     |stressLevel|tsPlacing          |
+--------+----------+-----------+-------------------+---------+-----------+-----------+-------------------+
|1       |movingGood|7          |2021-10-28 11:53:46|1        |placingGood|3          |2021-10-28 12:36:59|
|1       |movingGood|7          |2021-10-28 11:52:40|1        |placingGood|3          |2021-10-28 12:36:59|
|1       |movingGood|7          |2021-10-28 11:53:13|1        |placingGood|3          |2021-10-28 12:36:59|
|1       |movingGood|7          |2021-10-28 11:52:07|1        |placingGood|3          |2021-10-28 12:36:59|
|1       |movingGood|7          |2021-10-28 11:54:52|1        |placingGood|3          |2021-10-28 12:36:59|
+--------+----------+-----------+-------------------+---------+-----------+-----------+-------------------+
only showing top 5 rows



#### Discussion

Is this what we want?

Let's try to count how many joins we have here ...


In [43]:
spark.sql("SELECT idPlacing, tsPlacing, count(*) FROM sinkTable group by idPlacing, tsPlacing ORDER BY tsPlacing DESC").show(20,False) 

+---------+-------------------+--------+
|idPlacing|tsPlacing          |count(1)|
+---------+-------------------+--------+
|1        |2021-10-28 12:41:24|90      |
|2        |2021-10-28 12:41:24|89      |
|2        |2021-10-28 12:40:51|88      |
|1        |2021-10-28 12:40:51|89      |
|1        |2021-10-28 12:40:18|88      |
|2        |2021-10-28 12:40:18|87      |
|1        |2021-10-28 12:39:45|87      |
|2        |2021-10-28 12:39:45|86      |
|2        |2021-10-28 12:39:11|85      |
|1        |2021-10-28 12:39:11|86      |
|1        |2021-10-28 12:38:38|85      |
|2        |2021-10-28 12:38:38|84      |
|1        |2021-10-28 12:38:05|84      |
|2        |2021-10-28 12:38:05|83      |
|2        |2021-10-28 12:37:32|82      |
|1        |2021-10-28 12:37:32|83      |
|1        |2021-10-28 12:36:59|82      |
|2        |2021-10-28 12:36:59|81      |
|2        |2021-10-28 12:36:26|80      |
|1        |2021-10-28 12:36:26|81      |
+---------+-------------------+--------+
only showing top

**Far too many!** ... and growing :-(

O-: !!! ... and also **the state is growing** !!! :-O

In [ ]:
from IPython.display import clear_output
import json
while True:
    print(json.dumps(e3.lastProgress, indent=4))
    print(e3.status)
    time.sleep(1)
    clear_output(wait=True)

Monitor for a minute the field `"numRowsTotal"` in `"stateOperators"`.

**We need to add watermarks and a time constraint for state cleanup!**

In [44]:
e3.stop()

In [45]:
movingW_sdf = (roboticArm_sdf
                .withWatermark("ts","1 minute") # WATERMARK ADDED HERE
                .where("status='movingGood'")
                .withColumnRenamed("id","idMoving")
                .withColumnRenamed("ts","tsMoving")
               )

placingW_sdf = (roboticArm_sdf
                .withWatermark("ts","1 minute") # WATERMARK ADDED HERE
                .where("status='placingGood'")
                .withColumnRenamed("id","idPlacing")
                .withColumnRenamed("ts","tsPlacing")
               )

In [56]:
joinTC_sdf = (movingW_sdf.join(
  placingW_sdf, expr("""
    (idMoving == idPlacing) AND
    (tsPlacing > tsMoving ) AND
    (tsPlacing < tsMoving + interval 14 seconds )""" # TIME CONSTRAIN ADDED HERE (considering also that the time flows at half of the speed) !!!
    )))

In [57]:
e3TC = (joinTC_sdf
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

In [60]:
spark.sql("SELECT idPlacing, tsPlacing, count(*) FROM sinkTable group by idPlacing, tsPlacing ORDER BY tsPlacing DESC").show(20,False) # note, I change ts in tsTemp

+---------+-------------------+--------+
|idPlacing|tsPlacing          |count(1)|
+---------+-------------------+--------+
|1        |2021-10-28 12:50:49|1       |
|2        |2021-10-28 12:50:49|2       |
|2        |2021-10-28 12:50:16|2       |
|1        |2021-10-28 12:50:16|1       |
|2        |2021-10-28 12:49:43|2       |
|1        |2021-10-28 12:49:43|1       |
|1        |2021-10-28 12:49:09|1       |
|2        |2021-10-28 12:49:09|2       |
|2        |2021-10-28 12:48:36|2       |
|1        |2021-10-28 12:48:36|1       |
|1        |2021-10-28 12:48:03|1       |
|2        |2021-10-28 12:48:03|2       |
|2        |2021-10-28 12:47:29|2       |
|1        |2021-10-28 12:47:29|1       |
|1        |2021-10-28 12:46:56|1       |
|2        |2021-10-28 12:46:56|2       |
|2        |2021-10-28 12:46:23|2       |
|1        |2021-10-28 12:46:23|1       |
|2        |2021-10-28 12:45:50|2       |
|1        |2021-10-28 12:45:50|1       |
+---------+-------------------+--------+
only showing top

also notice that the state no longer grows

In [ ]:
from IPython.display import clear_output
import json
while True:
    print(json.dumps(e3TC.lastProgress, indent=4))
    print(e3TC.status)
    time.sleep(1)
    clear_output(wait=True)

In [61]:
e3TC.stop()

**How much should the watermark and the time constraint be?**

|watermark | time constraint | number of results per arm |reason                                         |
|----------|-----------------|-------------------|-----------------------------------------------|
|  any     |           <=10  |         0         | for each arm, there is no placing within less than 10 sec to a moving |   
|  any     |    >10 & < 14   |         1 or 0    | for one of the arms, there is 1 placing within 10-14 sec to a moving        |    
|  any     |    >14 & < 44   |         1         | for each arm, there is 1 placing within 10-43 sec to a moving                 
|  any     |    >=44 & <48   |         1 or 2    | for one of the arms, there are 2 placing within 44-47 sec to a moving                |   
|  any     |          >=48   |         2+        | for each of the arm, there are more than 2 placing within 48 or more sec to a moving                     |   

The watermark does not influence the answer because, in this case, the data arrive in order and without any delay, **However it is important to clean the state**.

### E3.2

> A continuous query that emits the events between a **pick (status==goodGrasped)** and a place (status==placingGood)

NOTE: I'm adding one more type of event, good grasped that should appear before any moving

In [63]:
grapsedW_sdf = (roboticArm_sdf
                .withWatermark("ts","1 minute") # WATERMARK ADDED HERE
                .where("status='goodGrasped'")
                .withColumnRenamed("id","idGrasped")
                .withColumnRenamed("ts","tsGrasped")
               )

movingW_sdf = (roboticArm_sdf
                .withWatermark("ts","1 minute") # WATERMARK ADDED HERE
                .where("status='movingGood'")
                .withColumnRenamed("id","idMoving")
                .withColumnRenamed("ts","tsMoving")
               )

placingW_sdf = (roboticArm_sdf
                .withWatermark("ts","1 minute") # WATERMARK ADDED HERE
                .where("status='placingGood'")
                .withColumnRenamed("id","idPlacing")
                .withColumnRenamed("ts","tsPlacing")
               )

In [65]:
joinGM_sdf = (grapsedW_sdf.join(
  movingW_sdf, expr("""
    (idGrasped == idMoving) AND
    (tsMoving > tsGrasped ) AND
    (tsMoving < tsGrasped + interval 3 seconds )""" # TIME CONSTRAIN ADDED HERE (considering also that the time flows at half of the speed) !!!
    )))

In [66]:
joinGM_sdf.printSchema()

root
 |-- idGrasped: string (nullable = true)
 |-- status: string (nullable = true)
 |-- stressLevel: integer (nullable = true)
 |-- tsGrasped: timestamp (nullable = true)
 |-- idMoving: string (nullable = true)
 |-- status: string (nullable = true)
 |-- stressLevel: integer (nullable = true)
 |-- tsMoving: timestamp (nullable = true)



In [67]:
placingW_sdf.printSchema()

root
 |-- idPlacing: string (nullable = true)
 |-- status: string (nullable = true)
 |-- stressLevel: integer (nullable = true)
 |-- tsPlacing: timestamp (nullable = true)



In [69]:
joinGMP_sdf = (joinGM_sdf.join(
  placingW_sdf, expr("""
    (idMoving == idPlacing) AND
    (tsPlacing > tsMoving ) AND
    (tsPlacing < tsMoving + interval 14 seconds )""" # TIME CONSTRAIN ADDED HERE (considering also that the time flows at half of the speed) !!!
    )))

In [70]:
e3_2 = (joinGMP_sdf
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

In [73]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsGrasped DESC").show(20,False) # note, I change ts in tsTemp

+---------+-----------+-----------+-------------------+--------+----------+-----------+-------------------+---------+-----------+-----------+-------------------+
|idGrasped|status     |stressLevel|tsGrasped          |idMoving|status    |stressLevel|tsMoving           |idPlacing|status     |stressLevel|tsPlacing          |
+---------+-----------+-----------+-------------------+--------+----------+-----------+-------------------+---------+-----------+-----------+-------------------+
|2        |goodGrasped|5          |2021-10-28 13:14:24|2       |movingGood|9          |2021-10-28 13:14:25|2        |placingGood|3          |2021-10-28 13:14:35|
|1        |goodGrasped|1          |2021-10-28 13:14:20|1       |movingGood|8          |2021-10-28 13:14:22|1        |placingGood|3          |2021-10-28 13:14:35|
|2        |goodGrasped|5          |2021-10-28 13:13:50|2       |movingGood|9          |2021-10-28 13:13:51|2        |placingGood|3          |2021-10-28 13:14:01|
|1        |goodGrasped|1    

In [74]:
e3_2.stop()

### E3.3

> A continuous query that emits **the average stress level** between a pick (status==goodGrasped) and a place (status==placingGood)

NOTE: the original question

In [75]:
grapsedFinal_sdf = (roboticArm_sdf
                .withWatermark("ts","1 minute") # WATERMARK ADDED HERE
                .where("status='goodGrasped'")
                .withColumnRenamed("stressLevel","stressLevelGrasped")
                .withColumnRenamed("id","idGrasped")
                .withColumnRenamed("ts","tsGrasped")
               )

movingFinal_sdf = (roboticArm_sdf
                .withWatermark("ts","1 minute") # WATERMARK ADDED HERE
                .where("status='movingGood'")
                .withColumnRenamed("stressLevel","stressLevelMoving")
                .withColumnRenamed("id","idMoving")
                .withColumnRenamed("ts","tsMoving")
               )

placingFinal_sdf = (roboticArm_sdf
                .withWatermark("ts","1 minute") # WATERMARK ADDED HERE
                .where("status='placingGood'")
                .withColumnRenamed("stressLevel","stressLevelPlacing")
                .withColumnRenamed("id","idPlacing")
                .withColumnRenamed("ts","tsPlacing")
               )

In [76]:
placingFinal_sdf.printSchema()

root
 |-- idPlacing: string (nullable = true)
 |-- status: string (nullable = true)
 |-- stressLevelPlacing: integer (nullable = true)
 |-- tsPlacing: timestamp (nullable = true)



In [77]:
joinFinal_sdf = (grapsedFinal_sdf.join(
  movingFinal_sdf, expr("""
    (idGrasped == idMoving) AND
    (tsMoving > tsGrasped ) AND
    (tsMoving < tsGrasped + interval 3 seconds )""" 
    )).join(
  placingFinal_sdf, expr("""
    (idMoving == idPlacing) AND
    (tsPlacing > tsMoving ) AND
    (tsPlacing < tsMoving + interval 14 seconds )"""
    ))
)

In [78]:
joinFinal_sdf.printSchema()

root
 |-- idGrasped: string (nullable = true)
 |-- status: string (nullable = true)
 |-- stressLevelGrasped: integer (nullable = true)
 |-- tsGrasped: timestamp (nullable = true)
 |-- idMoving: string (nullable = true)
 |-- status: string (nullable = true)
 |-- stressLevelMoving: integer (nullable = true)
 |-- tsMoving: timestamp (nullable = true)
 |-- idPlacing: string (nullable = true)
 |-- status: string (nullable = true)
 |-- stressLevelPlacing: integer (nullable = true)
 |-- tsPlacing: timestamp (nullable = true)



In [91]:
final_sdf = joinFinal_sdf.select(col("idGrasped").alias("id"), expr("(stressLevelGrasped+stressLevelMoving+stressLevelPlacing)/3 AS AVG_stressLevel"))

In [92]:
e3_3 = (final_sdf
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

In [94]:
spark.sql("SELECT * FROM sinkTable").show(20,False) # note, I change ts in tsTemp

+---+------------------+
|id |AVG_stressLevel   |
+---+------------------+
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
+---+------------------+
only showing top 20 rows



In [103]:
e3_3.stop()

## Think out of the box!!!

**Is the complexity of temporal joins acceptable? Is there any other solution that does not require them?**

In many cases, query answering is hard because the datamodel is **over simplified**.

We may go back to E1 problem and propose to change the model so to eliminate the need for a temporal join. A sequential number for the cycles of each arm would be enough to make the join deterministic. See [datagen2.ipynb](datagen1.ipynb) for the changes in the data generator.

In [95]:
roboticArmV2_schema = StructType([             ## <-- CHANGE HERE new name for the schema
    StructField("id", StringType(), True),
    StructField("cycle", IntegerType(), True), ## <-- CHANGE HERE new field 
    StructField("status", StringType(), True),
    StructField("stressLevel", IntegerType(), True),
    StructField("ts", TimestampType(), True)])

raw_roboticArmV2_df = (spark                   ## <-- CHANGE HERE new name for the df
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("startingOffsets", "earliest")
  .option("subscribe", "RoboticArmV2") ## <-- CHANGE HERE different topic
  .load())

roboticArmV2_sdf = (raw_roboticArmV2_df      ## <-- CHANGE HERE new name sdf
                      .select(from_json(col("value").cast("string"), roboticArmV2_schema).alias("value")) ## <-- CHANGE HERE new schema
                      .select("value.*"))

roboticArmV2_sdf.printSchema()

root
 |-- id: string (nullable = true)
 |-- cycle: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- stressLevel: integer (nullable = true)
 |-- ts: timestamp (nullable = true)



In [98]:
grapsedFinalCyc_sdf = (roboticArmV2_sdf
                .withWatermark("ts","1 minute") 
                .where("status='goodGrasped'")
                .withColumnRenamed("stressLevel","stressLevelGrasped")
                .withColumnRenamed("id","idGrasped")
                .withColumnRenamed("cycle","cycleGrasped")
               )

movingFinalCyc_sdf = (roboticArmV2_sdf
                .withWatermark("ts","1 minute") 
                .where("status='movingGood'")
                .withColumnRenamed("stressLevel","stressLevelMoving")
                .withColumnRenamed("id","idMoving")
                .withColumnRenamed("cycle","cycleMoving")
               )

placingFinalCyc_sdf = (roboticArmV2_sdf
                .withWatermark("ts","1 minute") 
                .where("status='placingGood'")
                .withColumnRenamed("stressLevel","stressLevelPlacing")
                .withColumnRenamed("id","idPlacing")
                .withColumnRenamed("cycle","cyclePlacing")
               )

In [99]:
joinFinalCyc_sdf = (grapsedFinalCyc_sdf.join(
  movingFinalCyc_sdf, expr("""
    (idGrasped == idMoving) AND
    (cycleGrasped == cycleMoving )""" ## <- CHANGE HERE  
    )).join(
  placingFinalCyc_sdf, expr("""
    (idMoving == idPlacing) AND
    (cycleMoving == cyclePlacing)""" ## <- CHANGE HERE 
    ))
)

In [ ]:
joinV2_sdf = (movingV2_sdf.join(
  placingV2_sdf, expr("""
    (idMoving == idPlacing) AND
    (cyclePlacing == cycleMoving )""" ## <- CHANGE HERE 
    )))

In [100]:
finalCyc_sdf = joinFinalCyc_sdf.select(col("idGrasped").alias("id"), expr("(stressLevelGrasped+stressLevelMoving+stressLevelPlacing)/3 AS AVG_stressLevel"))

In [104]:
e3V2 = (finalCyc_sdf
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

In [106]:
spark.sql("SELECT * FROM sinkTable").show(20,False) # note, I change ts in tsTemp

+---+------------------+
|id |AVG_stressLevel   |
+---+------------------+
|2  |5.666666666666667 |
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|2  |5.666666666666667 |
|1  |3.6666666666666665|
|2  |5.666666666666667 |
|1  |3.6666666666666665|
|2  |5.666666666666667 |
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|1  |3.6666666666666665|
|2  |5.666666666666667 |
|1  |3.6666666666666665|
|2  |5.666666666666667 |
|1  |3.6666666666666665|
|2  |5.666666666666667 |
|2  |5.666666666666667 |
|2  |5.666666666666667 |
|1  |3.6666666666666665|
|2  |5.666666666666667 |
+---+------------------+
only showing top 20 rows



**Much easier** !!

**REMEMBER**: modeling and querying are *two sides of the same coin*

We are not in a traditional RDBMS where modeling is done once for all by the DB administrator and queries must conform to "the model". We are in a setting where performance matters more than governance and chaning the model is often the only way to keep good performances.

In [107]:
e3V2.stop()

# E4

>A continuous query that returns the robotic arms that,
>
> * in less than 20 second (was 10 in EPL, but here the time passes at half of the speed),
> * picked a good while safely operating,
> * moved it while the controller was raising a warning, and
> * placed it while safely operating again.


In [108]:
goodGraspedSafely_sdf = (roboticArmV2_sdf
                .withWatermark("ts","1 minute")
                .where("status='goodGrasped' AND stressLevel < 7")
                .withColumnRenamed("id","idGrasped")
                .withColumnRenamed("cycle","cycleGrasped")
                .withColumnRenamed("stressLevel","stressLevelGrasped")
                .withColumnRenamed("ts","tsGrasped")
               )

movingWarning_sdf = (roboticArmV2_sdf
                .withWatermark("ts","1 minute")
                .where("status='movingGood' AND stressLevel > 6 AND stressLevel < 9")
                .withColumnRenamed("id","idMoving")
                .withColumnRenamed("cycle","cycleMoving")
                .withColumnRenamed("stressLevel","stressLevelMoving")
                .withColumnRenamed("ts","tsMoving")
               )

placingSafely_sdf = (roboticArmV2_sdf
                .withWatermark("ts","1 minute")
                .where("status='placingGood' AND stressLevel < 7")
                .withColumnRenamed("id","idPlacing")
                .withColumnRenamed("cycle","cyclePlacing")
                .withColumnRenamed("stressLevel","stressLevelPlacing")
                .withColumnRenamed("ts","tsPlacing")
               )

join1_sdf = (goodGraspedSafely_sdf.join(
    movingWarning_sdf, expr("""
    (idGrasped == idMoving) AND
    (cycleGrasped == cycleMoving)""" 
    )))

join2_sdf = (join1_sdf.join(
  placingSafely_sdf, expr("""
    (idMoving == idPlacing) AND
    (cyclePlacing == cycleMoving )""" 
    )))

within20sec = join2_sdf.where("tsPlacing <= tsGrasped + interval 20 seconds")

e4 = (within20sec
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

In [113]:
spark.sql("SELECT idGrasped AS ID, cyclePlacing AS cycle FROM sinkTable ORDER BY cyclePlacing DESC").show(5,False) 

+---+-----+
|ID |cycle|
+---+-----+
|1  |24   |
|1  |23   |
|1  |22   |
|1  |21   |
|1  |20   |
+---+-----+
only showing top 5 rows



indeed only the arm whose ID is 1

In [114]:
e4.stop()

# E5

> A continuous query that monitors the results of the previous one (i.e., E4) and counts how many times each robotic arm is present in the stream over a window of 20 seconds updating the counting every 4 seconds.

In [139]:
within20sec.printSchema()

root
 |-- idGrasped: string (nullable = true)
 |-- cycleGrasped: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- stressLevelGrasped: integer (nullable = true)
 |-- tsGrasped: timestamp (nullable = true)
 |-- idMoving: string (nullable = true)
 |-- cycleMoving: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- stressLevelMoving: integer (nullable = true)
 |-- tsMoving: timestamp (nullable = true)
 |-- idPlacing: string (nullable = true)
 |-- cyclePlacing: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- stressLevelPlacing: integer (nullable = true)
 |-- tsPlacing: timestamp (nullable = true)



In [132]:
e5 = (within20sec
                     .withWatermark("tsPlacing", "1 minutes")
                     .groupBy(window("tsPlacing", "40 seconds", "8 seconds"),"idGrasped")
                     .count()
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable") 
                     .start())

In [141]:
spark.sql("SELECT * FROM sinkTable ORDER BY window DESC").show(10,False)

+------------------------------------------+---------+-----+
|window                                    |idGrasped|count|
+------------------------------------------+---------+-----+
|[2021-10-28 14:00:40, 2021-10-28 14:01:20]|1        |1    |
|[2021-10-28 14:00:32, 2021-10-28 14:01:12]|1        |1    |
|[2021-10-28 14:00:24, 2021-10-28 14:01:04]|1        |2    |
|[2021-10-28 14:00:16, 2021-10-28 14:00:56]|1        |1    |
|[2021-10-28 14:00:08, 2021-10-28 14:00:48]|1        |1    |
|[2021-10-28 14:00:00, 2021-10-28 14:00:40]|1        |1    |
|[2021-10-28 13:59:52, 2021-10-28 14:00:32]|1        |2    |
|[2021-10-28 13:59:44, 2021-10-28 14:00:24]|1        |1    |
|[2021-10-28 13:59:36, 2021-10-28 14:00:16]|1        |1    |
|[2021-10-28 13:59:28, 2021-10-28 14:00:08]|1        |1    |
+------------------------------------------+---------+-----+
only showing top 10 rows



In [142]:
e5.stop()